In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import Normalize
import torch.nn.functional as F
import open_clip
from timm.data import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD

from helper import count_parameters

In [3]:
class CLIP(nn.Module):
    def __init__(self, name='ViT-L/14', pretrained='openai'):
        super().__init__()
        self.model = open_clip.create_model(name, pretrained=pretrained)
        self.model = self.model.eval().requires_grad_(False)
        self.img_resolution = self.model.visual.image_size[0]
        self.norm = Normalize(OPENAI_CLIP_MEAN, OPENAI_CLIP_STD)
        self.im_dim = self.txt_dim = self.model.ln_final.normalized_shape[0]

    @property
    def device(self) -> torch.device:
        return next(self.model.parameters()).device
    
    def encode_images(self, images: torch.Tensor, div255: bool = False) -> torch.Tensor :
        if div255: images = images.to(torch.float32) / 255
        images = F.interpolate(images, size = self.img_resolution, mode = "bicubic", align_corners=False)
        images = self.norm(images)
        image_features = self.model.encode_image(images)
        image_features = F.normalize(image_features, dim = -1)
        return image_features

    def encode_text(self, texts: list[str]) -> torch.Tensor:
        text = open_clip.tokenize(texts).to(self.device)
        text_features = self.model.encode_text(text)
        text_features = F.normalize(text_features, dim=-1)
        return text_features
    
    def forward(self, images: torch.Tensor, texts: list[str], div255: bool = False) -> torch.Tensor:
        assert len(images) == len(texts)
        image_features = self.encode_images(images, div255=div255)
        text_features = self.encode_text(texts)
        joint_features = torch.cat([image_features, text_features], 1)
        return joint_features

In [34]:
clip = CLIP()
image_features = clip.encode_images(torch.rand(2, 3, 224, 224))
image_features

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


tensor([[-0.0116,  0.0272,  0.0254,  ...,  0.0120, -0.0107, -0.0332],
        [-0.0126,  0.0286,  0.0246,  ...,  0.0149, -0.0142, -0.0318]])

In [5]:
clip = CLIP()
joint_features = clip(torch.rand(2, 3, 224, 224), ["cat", "dog"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(


In [6]:
joint_features.shape

torch.Size([2, 1536])

In [7]:
count_parameters(clip)
print("---")
del clip.model.visual
count_parameters(clip)

Total parameters: 427,616,513
Trainable parameters: 0
---
Total parameters: 123,650,305
Trainable parameters: 0


In [32]:
model = open_clip.create_model("ViT-L/14", pretrained='openai')
model = model.eval().requires_grad_(False)
img_resolution = model.visual.image_size[0]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


In [33]:
img_resolution

224

In [29]:
next(model.parameters()).device

device(type='cpu')

In [25]:
model.ln_final.normalized_shape[0]

768